In [1]:
from fnmatch import fnmatch
import json
import logging
import sys
from uuid import uuid4
from datetime import datetime, timezone

from tortoise import Tortoise
from tortoise.contrib.pydantic import pydantic_model_creator
from models.newsgroup import Newsgroup
from models.message import Message

await Tortoise.init(db_url="sqlite://db.sqlite3", modules={"models": ["__main__"]})

fmt = logging.Formatter(
    fmt="%(asctime)s - %(name)s:%(lineno)d - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
sh = logging.StreamHandler(sys.stdout)
sh.setLevel(logging.DEBUG)
sh.setFormatter(fmt)

# will print debug sql
logger_db_client = logging.getLogger("db_client")
logger_db_client.setLevel(logging.DEBUG)
logger_db_client.addHandler(sh)

logger_tortoise = logging.getLogger("tortoise")
logger_tortoise.setLevel(logging.DEBUG)
logger_tortoise.addHandler(sh)

In [2]:
Msg_Pyd = pydantic_model_creator(Message)
Nws_Pyd = pydantic_model_creator(Newsgroup)

In [12]:
gte_date = datetime(year=99, month=12, day=31, tzinfo=timezone.utc, hour=23, minute=34 ,second=45)
ms = await Message.filter(created_at__gte=gte_date).prefetch_related("newsgroup")  # .values("message_id", "newsgroup__name")
m = ms[1]

2022-05-31 10:48:06 - tortoise.db_client:135 - DEBUG - SELECT "reply_to","path","from","id","message_id","newsgroup_id","references","user_agent","body","x_ref","created_at","organization","subject" FROM "message" WHERE "created_at">='0099-12-31 23:34:45+00:00': None
2022-05-31 10:48:06 - tortoise.db_client:135 - DEBUG - SELECT "status","updated_at","default_subscribe","id","name","created_at","description" FROM "newsgroup" WHERE "id" IN (2,2,2,2,2,1,1,2,2,1,1,1) ORDER BY "name" ASC: None


In [13]:
m_pyd = await Msg_Pyd.from_tortoise_orm(m)

In [14]:
m_pyd.dict()

{'id': 40,
 'from_': 'Thomas Schmitt <t.e.schmitt@posteo.de>',
 'created_at': datetime.datetime(2022, 5, 16, 20, 23, 5, 62607, tzinfo=<UTC>),
 'subject': 'Re: Quick search',
 'message_id': '<369469a0-a12d-427d-9f05-d34a899df408@planetzorg.net>',
 'path': '!planetzorg.net',
 'references': '<73c5d02e-9b45-43c7-b71d-c6939048768f@planetzorg.net>',
 'reply_to': '',
 'organization': '',
 'x_ref': None,
 'user_agent': 'Pan/0.149 (Bellevue; 4c157ba git@gitlab.gnome.org:GNOME/pan.git)',
 'body': 'Ich weiß nicht, ich denke eher nicht.\n\n> Eine wunderbare Heiterkeit hat meine ganze Seele eingenommen, gleich den\n> süßen Frühlingsmorgen, die ich mit ganzem Herzen genieße. Ich bin allein\n> und freue mich meines Lebens in dieser Gegend, die für solche Seelen\n> geschaffen ist wie die meine. Ich bin so glücklich, mein Bester, so ganz\n> in dem Gefühle von ruhigem Dasein versunken, daß meine Kunst darunter\n> leidet. Ich könnte jetzt nicht zeichnen, nicht einen Strich, und bin nie\n> ein größerer Ma

In [24]:
await Message.filter(created_at__gte=gte_date).values_list("id", "from_", "newsgroup__name")

2022-05-31 10:52:10 - tortoise.db_client:135 - DEBUG - SELECT "message"."id" "0","message"."from" "1","message__newsgroup"."name" "2" FROM "message" LEFT OUTER JOIN "newsgroup" "message__newsgroup" ON "message__newsgroup"."id"="message"."newsgroup_id" WHERE "message"."created_at">='0099-12-31 23:34:45+00:00': None


[(39,
  'Thomas Schmitt <t.e.schmitt@posteo.de>',
  'germany.hessen.darmstadt-dieburg.dieburg'),
 (40,
  'Thomas Schmitt <t.e.schmitt@posteo.de>',
  'germany.hessen.darmstadt-dieburg.dieburg'),
 (41,
  'Thomas Schmitt <t.e.schmitt@posteo.de>',
  'germany.hessen.darmstadt-dieburg.dieburg'),
 (42,
  'Thomas Schmitt <t.e.schmitt@posteo.de>',
  'germany.hessen.darmstadt-dieburg.dieburg'),
 (43,
  'Thomas Schmitt <t.e.schmitt@posteo.de>',
  'germany.hessen.darmstadt-dieburg.dieburg'),
 (44,
  'Thomas Schmitt <t.e.schmitt@posteo.de>',
  'germany.hessen.darmstadt.darmstadt'),
 (45,
  'Thomas Schmitt <t.e.schmitt@posteo.de>',
  'germany.hessen.darmstadt.darmstadt'),
 (46,
  'Thomas Schmitt <t.e.schmitt@posteo.de>',
  'germany.hessen.darmstadt-dieburg.dieburg'),
 (47,
  'Thomas Schmitt <t.e.schmitt@posteo.de>',
  'germany.hessen.darmstadt-dieburg.dieburg'),
 (48,
  'Thomas Schmitt <t.e.schmitt@posteo.de>',
  'germany.hessen.darmstadt.darmstadt'),
 (49,
  'Thomas Schmitt <t.e.schmitt@posteo.de>'

In [22]:
await Message.filter(created_at__gte=gte_date).values("newsgroup__name")

2022-05-31 10:51:40 - tortoise.db_client:144 - DEBUG - SELECT "message__newsgroup"."name" "newsgroup__name" FROM "message" LEFT OUTER JOIN "newsgroup" "message__newsgroup" ON "message__newsgroup"."id"="message"."newsgroup_id" WHERE "message"."created_at">='0099-12-31 23:34:45+00:00': None


[{'newsgroup__name': 'germany.hessen.darmstadt-dieburg.dieburg'},
 {'newsgroup__name': 'germany.hessen.darmstadt-dieburg.dieburg'},
 {'newsgroup__name': 'germany.hessen.darmstadt-dieburg.dieburg'},
 {'newsgroup__name': 'germany.hessen.darmstadt-dieburg.dieburg'},
 {'newsgroup__name': 'germany.hessen.darmstadt-dieburg.dieburg'},
 {'newsgroup__name': 'germany.hessen.darmstadt.darmstadt'},
 {'newsgroup__name': 'germany.hessen.darmstadt.darmstadt'},
 {'newsgroup__name': 'germany.hessen.darmstadt-dieburg.dieburg'},
 {'newsgroup__name': 'germany.hessen.darmstadt-dieburg.dieburg'},
 {'newsgroup__name': 'germany.hessen.darmstadt.darmstadt'},
 {'newsgroup__name': 'germany.hessen.darmstadt.darmstadt'},
 {'newsgroup__name': 'germany.hessen.darmstadt.darmstadt'}]

In [25]:
m40 = await Message.get(id=40)

2022-05-31 10:41:41 - tortoise.db_client:135 - DEBUG - SELECT "message_id","organization","body","subject","newsgroup_id","from","user_agent","x_ref","references","created_at","reply_to","id","path" FROM "message" WHERE "id"=40 LIMIT 2: None


In [22]:
await m40.newsgroup

2022-05-31 10:41:26 - tortoise.db_client:135 - DEBUG - SELECT "updated_at","default_subscribe","status","created_at","description","id","name" FROM "newsgroup" WHERE "id"=2 ORDER BY "name" ASC LIMIT 1: None


Newsgroup <germany.hessen.darmstadt-dieburg.dieburg>

In [56]:
from tortoise.functions import Count, Max, Min
from tortoise.queryset import ValuesQuery

group_name = "germany.hessen.darmstadt-dieburg.dieburg"
current_group = await Newsgroup.get_or_none(name=group_name)

2022-05-18 12:05:04 - tortoise.db_client:135 - DEBUG - SELECT "status","id","name","description","created_at","default_subscribe","updated_at" FROM "newsgroup" WHERE "name"='germany.hessen.darmstadt-dieburg.dieburg' ORDER BY "name" ASC LIMIT 2: None


In [70]:
m = await Message.filter(newsgroup=current_group).order_by("id").first().prefetch_related("newsgroup")

2022-05-18 12:12:14 - tortoise.db_client:135 - DEBUG - SELECT "id","body","path","references","from","newsgroup_id","user_agent","message_id","organization","created_at","x_ref","subject","reply_to" FROM "message" WHERE "newsgroup_id"=2 ORDER BY "id" ASC LIMIT 1: None
2022-05-18 12:12:14 - tortoise.db_client:135 - DEBUG - SELECT "status","id","name","description","created_at","default_subscribe","updated_at" FROM "newsgroup" WHERE "id" IN (2) ORDER BY "name" ASC: None


In [68]:
await m.fetch_related("newsgroup")
m.newsgroup.name

2022-05-18 12:07:26 - tortoise.db_client:135 - DEBUG - SELECT "status","id","name","description","created_at","default_subscribe","updated_at" FROM "newsgroup" WHERE "id" IN (2) ORDER BY "name" ASC: None


'germany.hessen.darmstadt-dieburg.dieburg'

In [30]:
m.body

"Eine wunderbare Heiterkeit hat meine ganze Seele eingenommen, gleich den\nsüßen Frühlingsmorgen, die ich mit ganzem Herzen genieße. Ich bin allein\nund freue mich meines Lebens in dieser Gegend, die für solche Seelen\ngeschaffen ist wie die meine. Ich bin so glücklich, mein Bester, so ganz\nin dem Gefühle von ruhigem Dasein versunken, daß meine Kunst darunter\nleidet. Ich könnte jetzt nicht zeichnen, nicht einen Strich, und bin nie\nein größerer Maler gewesen als in diesen Augenblicken.\n\nWenn das liebe Tal um mich dampft, und die hohe Sonne an der Oberfläche\nder undurchdringlichen Finsternis meines Waldes ruht, und nur einzelne\nStrahlen sich in das innere Heiligtum stehlen, ich dann im hohen Grase am\nfallenden Bache liege, und näher an der Erde tausend mannigfaltige\nGräschen mir merkwürdig werden; wenn ich das Wimmeln der kleinen Welt\nzwischen Halmen, die unzähligen, unergründlichen Gestalten der Würmchen,\nder Mückchen näher an meinem Herzen fühle, und fühle die Gegenwart des\

In [3]:
msgs = await Message.filter(newsgroup__name=group_name)

2022-05-16 09:35:32 - tortoise.db_client:70 - DEBUG - Created connection <Connection(Thread-5, started 139931281245760)> with params: filename=db.sqlite3 journal_mode=WAL journal_size_limit=16384 foreign_keys=ON
2022-05-16 09:35:32 - tortoise.db_client:135 - DEBUG - SELECT "message"."id","message"."subject","message"."parent_id_id","message"."created_at","message"."references","message"."sender","message"."thread_id","message"."body","message"."message_id","message"."updated_at","message"."newsgroup_id" FROM "message" LEFT OUTER JOIN "newsgroup" "message__newsgroup" ON "message__newsgroup"."id"="message"."newsgroup_id" WHERE "message__newsgroup"."name"='schmiti.home.cars': None


In [4]:
print(f"count: {len(msgs)}, low: {min(msg.id for msg in msgs)}, high: {max(msg.id for msg in msgs)}")

count: 10, low: 37, high: 49


In [6]:
cmsg = msgs[2]
cmsg

<Message: 40>

In [14]:
await Message.filter(newsgroup__name=group_name, id__gt=38).order_by("id").first()

2022-05-16 09:49:15 - tortoise.db_client:135 - DEBUG - SELECT "message"."id","message"."subject","message"."parent_id_id","message"."created_at","message"."references","message"."sender","message"."thread_id","message"."body","message"."message_id","message"."updated_at","message"."newsgroup_id" FROM "message" LEFT OUTER JOIN "newsgroup" "message__newsgroup" ON "message__newsgroup"."id"="message"."newsgroup_id" WHERE "message__newsgroup"."name"='schmiti.home.cars' AND "message"."id">38 ORDER BY "message"."id" ASC LIMIT 1: None


<Message: 40>

In [4]:
groups = await Newsgroup.all()
for i in range(36):
    await Message.create(
        newsgroup=groups[i%len(groups)],
        from_="t.e.schmitt@gmail.com",
        message_id=f"<{uuid4()}@whatever.com>",
        subject="Need help painting the walls",
        body="Around May sometime would be great.",
        created_at=datetime.utcnow(),
        path="A"
    )

2022-05-16 21:56:01 - tortoise.db_client:135 - DEBUG - SELECT "created_at","description","name","default_subscribe","status","updated_at","id" FROM "newsgroup" ORDER BY "name" ASC: None


AttributeError: 'str' object has no attribute '_saved_in_db'

In [5]:
async for g in Newsgroup.all():
    print(g.__repr__())

2022-05-16 21:56:48 - tortoise.db_client:135 - DEBUG - SELECT "created_at","description","name","default_subscribe","status","updated_at","id" FROM "newsgroup" ORDER BY "name" ASC: None
Newsgroup <germany.hessen.darmstadt-dieburg.dieburg>
Newsgroup <germany.hessen.darmstadt.darmstadt>


In [ ]:
from tortoise.functions import Count, Max, Min

group_stats = (
        await Message.annotate(count=Count("id"), max=Max("id"), min=Min("id"))
        .group_by("newsgroup_id")
        .values(name="newsgroup__name", count="count", min="min", max="max")
    )
result_stats = [f"{g['name']} {g['max']} {g['min']}" for g in group_stats]

In [12]:
seeksell = await Newsgroup.get_or_none(name="seekandsell.hessen.darmstadt.darmstadt")

2022-04-28 17:06:05 - tortoise.db_client:135 - DEBUG - SELECT "description","id","status","default_subscribe","created_at","name","updated_at" FROM "newsgroup" WHERE "name"='seekandsell.hessen.darmstadt.darmstadt' ORDER BY "name" ASC LIMIT 2: None


In [13]:
await Message.get_or_none(id=35, newsgroup=seeksell)

2022-04-28 17:06:05 - tortoise.db_client:135 - DEBUG - SELECT "id","thread_id","subject","references","body","created_at","message_id","sender","newsgroup_id","updated_at","parent_id_id" FROM "message" WHERE "id"=35 AND "newsgroup_id"=5 LIMIT 2: None


<Message: 35>